In [1]:
from pathlib import Path
from glob import glob
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import KFold, GridSearchCV, cross_validate

from transformers import BertModel, BertConfig, BertTokenizer
from transformers import RobertaModel, RobertaConfig, RobertaTokenizer
from transformers import ElectraModel, ElectraConfig, ElectraTokenizer
from transformers import XLNetModel, XLNetTokenizer
import torch

In [2]:
RANDOM_STATE = 42

# Tweet Loading

Read raw tweet data from the "data" path.

In [3]:
raw_df = pd.read_excel("data/COVID19_Dataset-CM-ZB-complete with sources.xlsx")

# Generate Transformer Embeddings

We already have embeddings for DL, ICA, LDA, and NMF.  We now generate embeddings for a variety of modern transformer models.  The embedding method we're using for this process is to take the mean of the final layer of activations for the sentence.

In [4]:
def generate_mean_output_embeddings(text_list, model_definition):
    tokenizer = model_definition["tokenizer_module"].from_pretrained(model_definition["model_name"])
    model = model_definition["model_module"].from_pretrained(model_definition["model_name"])
    
    embeddings = []
    for text in text_list:
        inputs = tokenizer(text, return_tensors="pt")
        outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state[0]
        mean = torch.mean(last_hidden_states, 0)
        embeddings.append(mean.detach().numpy())
    
    return np.array(embeddings)

## Define models to use
Since HuggingFace uses a common framework for calling models, we can just define the settings for our Transformer experiments here.

In [5]:
model_definitions = [
    {
        "tokenizer_module": BertTokenizer,
        "model_module": BertModel,
        "model_name": "bert-base-uncased"
    },
    {
        "tokenizer_module": BertTokenizer,
        "model_module": BertModel,
        "model_name": "bert-base-cased"
    },
    {
        "tokenizer_module": BertTokenizer,
        "model_module": BertModel,
        "model_name": "bert-large-uncased"
    },
    {
        "tokenizer_module": BertTokenizer,
        "model_module": BertModel,
        "model_name": "bert-large-cased"
    },
    {
        "tokenizer_module": RobertaTokenizer,
        "model_module": RobertaModel,
        "model_name": "roberta-base"
    },
    {
        "tokenizer_module": RobertaTokenizer,
        "model_module": RobertaModel,
        "model_name": "roberta-large"
    },
    {
        "tokenizer_module": ElectraTokenizer,
        "model_module": ElectraModel,
        "model_name": "google/electra-base-discriminator"
    },
    {
        "tokenizer_module": ElectraTokenizer,
        "model_module": ElectraModel,
        "model_name": "google/electra-large-discriminator"
    }
]

## Generate embeddings
Generate embeddings for the models defined above.

In [6]:
for md in model_definitions:
    shortname = md['model_name'].split("/")[-1]
    output_path = "./data/transformer_embeddings"
    output_file = f"{output_path}/{shortname}.npy"
    Path(output_path).mkdir(parents=True, exist_ok=True)
    
    if Path(output_file).is_file():
        print(f"Embeddings {output_file} already exist. Skipping.")
    else:
        embeddings = generate_mean_output_embeddings(raw_df["Tweet"], md)
        np.save(output_file, embeddings)

Embeddings ./data/transformer_embeddings/bert-base-uncased.npy already exist. Skipping.
Embeddings ./data/transformer_embeddings/bert-base-cased.npy already exist. Skipping.
Embeddings ./data/transformer_embeddings/bert-large-uncased.npy already exist. Skipping.
Embeddings ./data/transformer_embeddings/bert-large-cased.npy already exist. Skipping.
Embeddings ./data/transformer_embeddings/roberta-base.npy already exist. Skipping.
Embeddings ./data/transformer_embeddings/roberta-large.npy already exist. Skipping.
Embeddings ./data/transformer_embeddings/electra-base-discriminator.npy already exist. Skipping.
Embeddings ./data/transformer_embeddings/electra-large-discriminator.npy already exist. Skipping.


# Register Embedding Files
Register the embedding files to use in the experiments below.

In [7]:
embedding_files = glob("./data/latent_var_embeddings/*.npy")
embedding_files.extend(glob("./data/transformer_embeddings/*.npy"))

In [8]:
embedding_files

['./data/latent_var_embeddings/tweet_embed_NMF.npy',
 './data/latent_var_embeddings/tweet_embed_LDA.npy',
 './data/latent_var_embeddings/tweet_embed_DL.npy',
 './data/latent_var_embeddings/tweet_embed_ICA.npy',
 './data/transformer_embeddings/electra-base-discriminator.npy',
 './data/transformer_embeddings/roberta-base.npy',
 './data/transformer_embeddings/electra-large-discriminator.npy',
 './data/transformer_embeddings/bert-base-cased.npy',
 './data/transformer_embeddings/roberta-large.npy',
 './data/transformer_embeddings/bert-base-uncased.npy',
 './data/transformer_embeddings/bert-large-cased.npy',
 './data/transformer_embeddings/bert-large-uncased.npy']

# Experiments

Train SVM using nested 5-fold cross validation to select hyperparameters for each of our embeddings.

In [9]:
def experiment(X, y):
    svr = svm.SVC(kernel="rbf")
    param_grid = {"C": [0.1, 1, 10, 100], "gamma": [1, 0.1, 0.01, 0.001]}

    inner_cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    outer_cv = KFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

    clf = GridSearchCV(estimator=svr, param_grid=param_grid, cv=inner_cv, refit=True)
    nested_scores = cross_validate(clf, X=X, y=y, cv=outer_cv, scoring=['accuracy', 'precision', 'recall', 'f1'], n_jobs=-1)
    return nested_scores

In [10]:
labels = raw_df["Is_Unreliable"].to_numpy()

complete_scores = {}

for file in embedding_files:
    embedding = np.load(file)
    exp_scores = experiment(embedding, labels)
    complete_scores[file] = exp_scores

In [11]:
result_df = pd.DataFrame()
for file, scores in complete_scores.items():
    result = {}
    result["file"] = file.split("/")[-1]
    
    for score_name, score in scores.items():
        result[score_name] = np.mean(score)
        
    result_df = result_df.append(result, ignore_index=True)

In [12]:
result_df

file   fit_time  score_time  test_accuracy  \
0               tweet_embed_NMF.npy   6.868492    0.014528       0.844643   
1               tweet_embed_LDA.npy   7.591569    0.017777       0.801786   
2                tweet_embed_DL.npy   7.883789    0.015654       0.867857   
3               tweet_embed_ICA.npy   6.743203    0.016039       0.853571   
4    electra-base-discriminator.npy  19.655761    0.032180       0.866071   
5                  roberta-base.npy  17.919648    0.029302       0.882143   
6   electra-large-discriminator.npy  26.340496    0.040517       0.846429   
7               bert-base-cased.npy  19.432056    0.041690       0.871429   
8                 roberta-large.npy  24.274658    0.037470       0.885714   
9             bert-base-uncased.npy  19.293456    0.030925       0.857143   
10             bert-large-cased.npy  25.712156    0.038271       0.867857   
11           bert-large-uncased.npy  24.965059    0.035771       0.873214   

     test_f1  test_precision  test_recall  
0   0.849378        0.827195     0.877052  
1   0.813359        0.770809     0.869105  
2   0.871553        0.834320     0.913011  
3   0.857524        0.826761     0.892817  
4   0.866440        0.876675     0.862346  
5   0.882419        0.889162     0.879060  
6   0.844821        0.853560     0.837753  
7   0.873175        0.867536     0.882588  
8   0.885508        0.886276     0.887609  
9   0.861235        0.840694     0.886474  
10  0.870288        0.857928     0.889483  
11  0.875651        0.858868     0.894077

# Formatting Table
Format the table for ease of display

In [13]:
formatted_df = result_df.copy().round(3)

In [14]:
formatted_df = formatted_df.rename(columns={"file": "File", "test_accuracy": "Accuracy", "test_f1": "F1", "test_precision": "Precision", "test_recall": "Recall"})

In [15]:
file_method_mapping = {
    'tweet_embed_NMF.npy': "NMF",
    'tweet_embed_LDA.npy': "LDA",
    'tweet_embed_DL.npy': "DL",
    'tweet_embed_ICA.npy': "ICA",
    'electra-base-discriminator.npy': "ELECTRA-Base",
    'roberta-base.npy': "RoBERTa-Base",
    'electra-large-discriminator.npy': "ELECTRA-Large",
    'bert-base-cased.npy': "BERT-Base-Cased",
    'roberta-large.npy': "RoBERTa-Large",
    'bert-base-uncased.npy': "BERT-Base-Uncased",
    'bert-large-cased.npy': "BERT-Large-Cased",
    'bert-large-uncased.npy': "BERT-Large-Uncased"
}

In [16]:
formatted_df["Method"] = formatted_df["File"].map(file_method_mapping)

In [17]:
formatted_df = formatted_df[["Method", "Accuracy", "Recall", "Precision", "F1"]]

In [18]:
formatted_df = formatted_df.set_index("Method")

In [19]:
formatted_df

Accuracy  Recall  Precision     F1
Method                                                
NMF                    0.845   0.877      0.827  0.849
LDA                    0.802   0.869      0.771  0.813
DL                     0.868   0.913      0.834  0.872
ICA                    0.854   0.893      0.827  0.858
ELECTRA-Base           0.866   0.862      0.877  0.866
RoBERTa-Base           0.882   0.879      0.889  0.882
ELECTRA-Large          0.846   0.838      0.854  0.845
BERT-Base-Cased        0.871   0.883      0.868  0.873
RoBERTa-Large          0.886   0.888      0.886  0.886
BERT-Base-Uncased      0.857   0.886      0.841  0.861
BERT-Large-Cased       0.868   0.889      0.858  0.870
BERT-Large-Uncased     0.873   0.894      0.859  0.876

In [20]:
print(formatted_df.to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &  Accuracy &  Recall &  Precision &     F1 \\
Method             &           &         &            &        \\
\midrule
NMF                &     0.845 &   0.877 &      0.827 &  0.849 \\
LDA                &     0.802 &   0.869 &      0.771 &  0.813 \\
DL                 &     0.868 &   0.913 &      0.834 &  0.872 \\
ICA                &     0.854 &   0.893 &      0.827 &  0.858 \\
ELECTRA-Base       &     0.866 &   0.862 &      0.877 &  0.866 \\
RoBERTa-Base       &     0.882 &   0.879 &      0.889 &  0.882 \\
ELECTRA-Large      &     0.846 &   0.838 &      0.854 &  0.845 \\
BERT-Base-Cased    &     0.871 &   0.883 &      0.868 &  0.873 \\
RoBERTa-Large      &     0.886 &   0.888 &      0.886 &  0.886 \\
BERT-Base-Uncased  &     0.857 &   0.886 &      0.841 &  0.861 \\
BERT-Large-Cased   &     0.868 &   0.889 &      0.858 &  0.870 \\
BERT-Large-Uncased &     0.873 &   0.894 &      0.859 &  0.876 \\
\bottomrule
\end{tabular}

